In [2]:
from pandas import DataFrame
import json
data = json.load(open("../data/MultiWoz2.json"))


In [4]:
data[0]

{'dialogue_id': 'PMUL0698.json',
 'services': ['restaurant', 'train'],
 'turns': [{'frames': [{'actions': [],
     'service': 'restaurant',
     'slots': [{'exclusive_end': 71,
       'slot': 'restaurant-food',
       'start': 64,
       'value': 'chinese'}],
     'state': {'active_intent': 'find_restaurant',
      'requested_slots': [],
      'slot_values': {'restaurant-area': ['centre'],
       'restaurant-food': ['chinese']}}},
    {'actions': [],
     'service': 'taxi',
     'slots': [],
     'state': {'active_intent': 'NONE',
      'requested_slots': [],
      'slot_values': {}}},
    {'actions': [],
     'service': 'train',
     'slots': [],
     'state': {'active_intent': 'NONE',
      'requested_slots': [],
      'slot_values': {}}},
    {'actions': [],
     'service': 'bus',
     'slots': [],
     'state': {'active_intent': 'NONE',
      'requested_slots': [],
      'slot_values': {}}},
    {'actions': [],
     'service': 'police',
     'slots': [],
     'state': {'active_inte

In [3]:
all_dic = dict()

for i in range(len(data)):
    chat = data[i]
    turns = chat['turns']
    for j in range(len(turns)):
        turn = turns[j]
        if turn['speaker'] == 'SYSTEM':
            continue
        frames = turn['frames']
        for k in range(len(frames)):
            frame = frames[k]
            state = frame['state']
            active_intent = state['active_intent']
            if (active_intent=='NONE'):
                continue
            if active_intent not in all_dic:
                all_dic[active_intent] = dict()
            keys = str(state['slot_values'].keys())
            keys = keys.replace('dict_keys([\'','')
            keys = keys.replace('dict_keys([])','[]')
            keys = keys.replace('\',','')
            keys = keys.replace('\'','')
            keys = keys.replace('])','')
            keys = keys.replace(' ',' | ')

            if keys not in all_dic[active_intent]:
                all_dic[active_intent][keys] = 1
            else:
                all_dic[active_intent][keys]+=1

                
for key in all_dic:
    all_dic[key]['TOTAL_VARIATIONS'] = len(all_dic[key].keys())
all_dic
            

{'find_restaurant': {'restaurant-area | restaurant-food': 79,
  'restaurant-area | restaurant-pricerange': 65,
  'restaurant-area | restaurant-food | restaurant-pricerange': 152,
  'restaurant-pricerange': 11,
  'restaurant-food | restaurant-pricerange': 67,
  'restaurant-food | restaurant-name | restaurant-pricerange': 14,
  '[]': 94,
  'restaurant-food': 17,
  'restaurant-area | restaurant-food | restaurant-name | restaurant-pricerange': 30,
  'restaurant-name': 59,
  'restaurant-area | restaurant-food | restaurant-name': 4,
  'restaurant-area': 13,
  'restaurant-area | restaurant-name | restaurant-pricerange': 7,
  'restaurant-area | restaurant-name': 3,
  'restaurant-area | restaurant-bookday | restaurant-name | restaurant-pricerange': 1,
  'restaurant-area | restaurant-bookday | restaurant-booktime | restaurant-name | restaurant-pricerange': 1,
  'restaurant-name | restaurant-pricerange': 1,
  'TOTAL_VARIATIONS': 17},
 'find_train': {'train-day | train-leaveat': 12,
  'train-day |

In [8]:
my_actions = set()
acts = set()
file = open("results.txt","w")


for i in range(len(data)):
    chat = data[i]
    turns = chat['turns']
    for j in range(len(turns)):
        turn = turns[j]
        if turn['speaker'] == 'SYSTEM':
            continue
        frames = turn['frames']
        for k in range(len(frames)):
            frame = frames[k]
            state = frame['state']
            active_intent = state['active_intent']
            if (active_intent=='NONE'):
                continue    
            for key in state['slot_values'].keys():
                pair = active_intent + key[0].upper() + key[1:]
                if pair not in my_actions:
                    my_actions.add(pair)
                    acts.add((active_intent,key)) 

file.write("(define (domain dialog)\n")
file.write("(:requirements :strips :fluents :durative-actions :timed-initial-literals :typing :conditional-effects :negative-preconditions :duration-inequalities :equality)\n")                    
                    
file.write("(:types\n")
types_set = set()
for intent,act in acts:
    if act not in types_set:
        types_set.add(act)
        f,s = act.split('-')
        file.write("    " + act + " - object \n")
file.write(")\n") 

                    
file.write("\n(:predicates\n")                     
for intent,act in acts:
    f,s = act.split('-')
    file.write("    ("+intent.split("_")[0]+"_"+ act +" ?"+f[0].upper() + s[0].upper() + " - " + act + ")\n")
file.write(")\n\n")                     

for active_intent,key in acts:
    pair = active_intent + key[0].upper() + key[1:]
    f,s = key.split('-')
    file.write("(:action " + pair + "\n")
    file.write("        :parameters (?" + f[0].upper() + s[0].upper() + " - " + key + ")\n")
    file.write("        :precondition ()\n")
    file.write("        :effect (" + active_intent.split('_')[0] + "_"+ key +" ?"+f[0].upper() + s[0].upper() +"))\n")
file.write(")\n")   

    
    
print("Done!")
print("acts: "+str(len(acts)))
print("actions: "+str(len(my_actions)))

file.close()
  






Done!
acts: 49
actions: 49
